In [2]:
import numpy as np
import cv2 as cv
from skimage import data, io, filters


In [ ]:
image = data.coins()
# ... or any other NumPy array!
edges = filters.sobel(image)
io.imshow(edges)
io.show()